#### proprocess data

In [10]:
import os
import sys
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import permutations
from multiprocessing import Pool

In [9]:
expre_file = "/data5/galaxy/project/expression/stringtie/Total-RPKM.txt"
m6a_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/permutation/"
expre_dir = "/data5/galaxy/project/expression/DESeq2/DESeq2_2/"
result_dir = "/data5/galaxy/project/expre_vs_m6a/expre_vs_m6a_2"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
# methylation level only have these tissue data.
# subdir_list = ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
subdir_list = [os.path.basename(x).split(".csv")[0] for x in glob.glob("%s/*.csv" % expre_dir)]
for sub_dir in subdir_list:
    print("##################################\n%s" % sub_dir)
    df_m6a_up, df_m6a_common, df_m6a_down = class_m6a_by_diffPeak(os.path.join(m6a_dir, sub_dir))
#     print(df_m6a_up.head())
#     sys.exit(0)
    df_m6a = pd.concat([df_m6a_up, df_m6a_common, df_m6a_down]).dropna()
    ###
    print("############ expression ##########")
    df_gene_up, df_gene_down = class_gene_by_diffexpre(os.path.join(expre_dir, "%s.csv" % sub_dir))
    df_gene_up = df_gene_up.merge(df_m6a, on="name", how="left").dropna()
    df_gene_down = df_gene_down.merge(df_m6a, on="name", how="left").dropna()
#     print(len(df_gene_up.merge(df_gene_down, on="name", how="left").dropna()))
#     print(df_gene_up.head())
    statistic_proportion(sub_dir, df_m6a_up, df_m6a_common, df_m6a_down, df_gene_up, df_gene_down)

##################################
pla_vs_kid
2553 2248
2769 2288
4273 4026
############ expression ##########
##################################
pla_vs_mus
2160 1787
3365 2891
3244 3047
############ expression ##########
##################################
liver_vs_brain
6970 6250
1990 1573
1054 1001
############ expression ##########
##################################
sto_vs_lung
1184 997
1097 979
7767 7334
############ expression ##########
##################################
brain_vs_lung
1347 1039
6335 5681
1585 1501
############ expression ##########
##################################
liver_vs_kid
3856 3549
2362 1970
2879 2715
############ expression ##########
##################################
brain_vs_heart
1847 1478
5239 4824
2028 1907
############ expression ##########
##################################
liver_vs_heart
3791 3280
2231 1979
3212 3043
############ expression ##########
##################################
sto_vs_liver
3034 2628
2446 2119
3976 3757
############ expre

In [2]:
def statistic_proportion(sub_dir, df_m6a_up, df_m6a_common, df_m6a_down, df_gene_up, df_gene_down):
#     print("up")
    each_type_expre(df_gene_up, "gene_up", sub_dir)
#     print("down")
    each_type_expre(df_gene_down, "gene_down", sub_dir)
    
def each_type_expre(df, gene_type, sub_dir):
    total = len(df["name"].drop_duplicates())
    m6a_up = df[df["name"].isin(df_m6a_up["name"])]["name"].drop_duplicates()
    result_file = os.path.join(result_dir, "%s-%s-%s.txt" % (sub_dir, gene_type, "m6a_up"))
    write_file(result_file, m6a_up)
    #
    m6a_common = df[df["name"].isin(df_m6a_common["name"])]["name"].drop_duplicates()
    result_file = os.path.join(result_dir, "%s-%s-%s.txt" % (sub_dir, gene_type, "m6a_common"))
    write_file(result_file, m6a_common)
    #
    m6a_down = df[df["name"].isin(df_m6a_down["name"])]["name"].drop_duplicates()
    result_file = os.path.join(result_dir, "%s-%s-%s.txt" % (sub_dir, gene_type, "m6a_down"))
    write_file(result_file, m6a_down)
#     up_prop, common_prop, down_prop = (m6a_up / total), (m6a_common / total), (m6a_down/ total)
#     return m6a_up, m6a_common, m6a_down
#     print("m6a up\tm6a common\tm6a down")
#     print("%f\t%f\t%f" % (up_prop, common_prop, down_prop))

def write_file(result_file, names):
    with open(result_file, 'w') as f:
        f.writelines([("%s\n" % x.strip()) for x in names])

In [3]:
def class_m6a_by_diffPeak(work_dir):
    os.chdir(work_dir)
    df_total_exp = get_expre()
    m6a_up = pd.read_table("diff_peak_c3.0_cond1_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
    m6a_common = pd.read_table("diff_peak_c3.0_common_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
    m6a_down = pd.read_table("diff_peak_c3.0_cond2_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
    # m6a score 计算方式
    df_m6a_up = m6a_up.groupby(["name"]).mean().reset_index()
    df_m6a_common = m6a_common.groupby(["name"]).mean().reset_index()
    df_m6a_down = m6a_down.groupby(["name"]).mean().reset_index()
    df_m6a_up, df_m6a_down, df_m6a_common = filter_intersect(df_m6a_up, df_m6a_down, df_m6a_common)
    # filter by expression
    sub_dir = work_dir.split("/")[-1]
    tissue_1, tissue_2 = sub_dir.split("_")[0], sub_dir.split("_")[-1]
    df_m6a_up = filter_by_exp(df_total_exp, df_m6a_up, tissue_1, tissue_2)
    df_m6a_down = filter_by_exp(df_total_exp, df_m6a_down, tissue_1, tissue_2)
    df_m6a_common = filter_by_exp(df_total_exp, df_m6a_common, tissue_1, tissue_2)
    return df_m6a_up, df_m6a_common, df_m6a_down

def get_expre():
    df = pd.read_table(expre_file, sep="\t")
    df.columns = ["gene", "brain", "heart", "kid", "liver", "pla", "lung", "mus", "sto"]
    df["name"] = df["gene"].str.split(".").str[0]
    return df

def filter_by_exp(df_total_exp, df, tissue_1, tissue_2):
    df_expre_1 = df_total_exp[["name", tissue_1]].set_index("name")
    df_expre_2 = df_total_exp[["name", tissue_2]].set_index("name")
    names = df.name
    clean_names = []
    for name in names:
        if (name in df_expre_1.index) and (name in df_expre_2.index):
#         print(df_expre_1.head())
            if (df_expre_1.loc[name, tissue_1] != 0.0) & (df_expre_2.loc[name, tissue_2] != 0.0):
                clean_names.append(name)
    df_clean = df[df["name"].isin(clean_names)]
    print(len(df), len(df_clean))
    return df_clean
    
def filter_intersect(df_up, df_down, df_remain):
#     print("before", len(df_up), len(df_down), len(df_remain))
    up_genes, down_genes, remain_genes = set(df_up["name"]), set(df_down["name"]), set(df_remain["name"])
    uniq_up = up_genes - down_genes - remain_genes
    uniq_down = down_genes - up_genes - remain_genes
    uniq_remain = remain_genes - up_genes - down_genes
    df_up, df_down, df_remain = select_uniq(uniq_up, df_up), select_uniq(uniq_down, df_down), select_uniq(uniq_remain, df_remain)
#     print("after", len(uniq_up), len(uniq_down), len(uniq_remain))
#     print("after", len(df_up), len(df_down), len(df_remain))
    return df_up, df_down, df_remain
    
def select_uniq(query_list, df):
    df = df[df["name"].isin(query_list)]
    return df

In [4]:
def format_gene_name(df):
    df["name"] = df["Unnamed: 0"].str.split(".").str[0]
#     df = df.set_index(["name"])
    del df["Unnamed: 0"]
    return df

def class_gene_by_diffexpre(DESeq2_file):
    df = pd.read_csv(DESeq2_file)
    df = df[(abs(df.log2FoldChange) > 1) & (df.padj < 0.05)]
    df_up, df_down = df.copy()[df.log2FoldChange > 0], df.copy()[df.log2FoldChange < 0]
#     print(len(df_up), len(df_down))
    return format_gene_name(df_up), format_gene_name(df_down)

In [5]:
## 2.we find the Spearman rho between a row of expression data and each row of methylation data, and we keep the row of methylation data that produces the most-negative rho value
def filter_each_row_by_expression(df, df_gene_up, df_gene_down):
#     print(len(df))
    df_pos, df_neg = df[df.logFC >0], df[df.logFC <0]
#     print(len(df_pos), len(df_neg))
#     print(df_pos.head())
#     print(df_gene_up.head())
    up_genes, down_genes, total_genes = df_gene_up.name, df_gene_down.name, list(set(df["name"].tolist()))
#     remain_genes = [x for x in total_genes if (x not in up_genes) and (x not in down_genes)]
    df_pos = df_pos[df_pos["name"].isin(down_genes)]
    df_neg = df_neg[df_neg["name"].isin(up_genes)]
#     print(len(df_pos), len(df_neg))
    df_coordinate = pd.concat([df_pos, df_neg]).dropna()
#     df_unchange = df[df["name"].isin(remain_genes)]
#     print(len(df), len(df_coordinate), len(df_unchange))
    return df_coordinate